In [1]:
import tensorflow as tf

In [2]:
from reader import create_inputs

In [3]:
image, label = create_inputs()

In [4]:
queue = tf.FIFOQueue(256, ['uint8', 'uint8'])

In [5]:
enqueue = queue.enqueue([image, label])

In [6]:
batch_input = queue.dequeue()

In [7]:
coord = tf.train.Coordinator()

In [8]:
qr = tf.train.QueueRunner(queue, [enqueue])

In [9]:
sess = tf.Session()

In [10]:
qr.create_threads(sess, coord=coord, start=True)

[<Thread(Thread-4, started 139720109582080)>,
 <Thread(Thread-5, started 139720101189376)>]

In [11]:
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [12]:
label_tensor = batch_input[1]

In [13]:
label_tensor = tf.reshape(label_tensor, [-1])

In [14]:
b = tf.greater(label_tensor, 0)

In [15]:
filter_label_tensor = tf.boolean_mask(label_tensor, b)

In [16]:
image_tensor = batch_input[0]

In [17]:
image_tensor = tf.reshape(image_tensor, [-1, 3])

In [18]:
tf.boolean_mask(image_tensor, b)

<tf.Tensor 'boolean_mask_1/Gather:0' shape=(?, 3) dtype=uint8>

In [19]:
b.get_shape()

TensorShape([Dimension(None)])

In [12]:
import matplotlib.pyplot as plt

In [134]:
d1 = sess.run(batch_input)

In [135]:
#plt.imshow(d1[0])

In [136]:
#plt.show()

In [137]:
d1[1].shape[0]

63640

In [17]:
d1[1].shape[0] == d1[0].shape[0] * d1[0].shape[1]

True